## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
for i in application_df.columns:
    print(i, len(application_df[i].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_totals = application_df['APPLICATION_TYPE'].value_counts()
app_type_totals

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_totals[app_type_totals < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.head(30)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
Name: CLASSIFICATION, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 700].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)
numeric_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = numeric_df['IS_SUCCESSFUL']
X = numeric_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 315       
                                                                 
 dense_1 (Dense)             (None, 4)                 32        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 352
Trainable params: 352
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 6s 3ms/step - loss: 0.6171 - accuracy: 0.6749
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5669 - accuracy: 0.7268
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7305
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5530 - accuracy: 0.7323

804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5510 - accuracy: 0.7316
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7323
Epoch 10/100
794/804 [============================>.] - ETA: 0s - loss: 0.5498 - accuracy: 0.7331

804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7328
Epoch 11/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7325
Epoch 12/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7320
Epoch 13/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7323
Epoch 14/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7328
Epoch 15/100
771/804 [===========================>..] - ETA: 0s - loss: 0.5487 - accuracy: 0.7326

804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7331
Epoch 16/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7326
Epoch 17/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7325
Epoch 18/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7334
Epoch 19/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7320
Epoch 20/100
801/804 [============================>.] - ETA: 0s - loss: 0.5472 - accuracy: 0.7328

804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7328
Epoch 21/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7327
Epoch 22/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7326
Epoch 23/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 24/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7330
Epoch 25/100
775/804 [===========================>..] - ETA: 0s - loss: 0.5462 - accuracy: 0.7329

...

804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7325
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7331
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7338
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 0.7332
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7328
Epoch 85/100
779/804 [============================>.] - ETA: 0s - loss: 0.5435 - accuracy: 0.7336

804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7333
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7341
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5438 - accuracy: 0.7332
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7334
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7331
Epoch 90/100
780/804 [============================>.] - ETA: 0s - loss: 0.5436 - accuracy: 0.7335

804/804 [==============================] - 2s 2ms/step - loss: 0.5436 - accuracy: 0.7335
Epoch 91/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7329
Epoch 92/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5435 - accuracy: 0.7331
Epoch 93/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5435 - accuracy: 0.7335
Epoch 94/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5437 - accuracy: 0.7336
Epoch 95/100
801/804 [============================>.] - ETA: 0s - loss: 0.5431 - accuracy: 0.7329

804/804 [==============================] - 2s 2ms/step - loss: 0.5436 - accuracy: 0.7325
Epoch 96/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5435 - accuracy: 0.7332
Epoch 97/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7333
Epoch 98/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7331
Epoch 99/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7329
Epoch 100/100
781/804 [============================>.] - ETA: 0s - loss: 0.5432 - accuracy: 0.7332

804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7331


In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7257 - 356ms/epoch - 1ms/step
Loss: 0.5555788278579712, Accuracy: 0.7257142663002014


In [16]:
# Export our model to HDF5 file
nn.save("nn_original.h5")